In [ ]:
!pip install mlxtend==0.18.0

In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import EnsembleVoteClassifier, StackingCVClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/content/train_credit_risk.csv')
df.head()

,loan_amnt,funded_amnt,pymnt_plan,grade,sub_grade_num,short_emp,emp_length_num,home_ownership,dti,purpose,payment_inc_ratio,delinq_2yrs,delinq_2yrs_zero,inq_last_6mths,last_delinq_none,last_major_derog_none,open_acc,pub_rec,pub_rec_zero,revol_util,bad_loans
0,5000,5000,n,B,0.4,0,11,RENT,27.65,credit_card,8.14350,0.0,1.0,1.0,1,1,3.0,0.0,1.0,83.7,0
1,2400,2400,n,C,1.0,0,11,RENT,8.72,small_business,8.25955,0.0,1.0,2.0,1,1,2.0,0.0,1.0,98.5,0
2,5000,5000,n,A,0.8,0,4,RENT,11.20,wedding,5.21533,0.0,1.0,3.0,1,1,9.0,0.0,1.0,28.3,0
3,3000,3000,n,E,0.2,0,10,RENT,5.35,car,2.73575,0.0,1.0,2.0,1,1,4.0,0.0,1.0,87.5,0
4,5600,5600,n,F,0.4,0,5,OWN,5.55,small_business,4.57170,0.0,1.0,2.0,1,1,11.0,0.0,1.0,32.6,1


In [ ]:
X = df.drop('bad_loans', axis=1)
y = df['bad_loans']
y.value_counts()

0    66266
1    15472
Name: bad_loans, dtype: int64

In [ ]:
LE = LabelEncoder()
for c in X.columns:
  if X[c].dtypes=='object':
    X[c] = LE.fit_transform(X[c])
X = X.fillna(0)

In [ ]:
from mlxtend.classifier import EnsembleVoteClassifier

clf1 = LogisticRegression(random_state=1)
clf2 = DecisionTreeClassifier(random_state=1, class_weight='balanced')
clf3 = GaussianNB()

vclf = EnsembleVoteClassifier(clfs=[clf1, clf2, clf3], weights=[3,2,4], voting='soft')
scores = cross_val_score(vclf, X, y, cv=5, scoring='f1')
print(scores.mean())

nan


In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, y, test_size=0.2)

In [ ]:
vclf.fit(Xtrain, Ytrain)
pred = vclf.predict(Xtest)
print(accuracy_score(Ytest, pred))
print(confusion_matrix(Ytest, pred))

0.9333333333333333
[[12  0  0]
 [ 0  8  2]
 [ 0  0  8]]


In [ ]:
from sklearn.ensemble import BaggingClassifier
bclf = BaggingClassifier(base_estimator=clf3, n_estimators=20, max_samples=0.9, random_state=2020)
scores = cross_val_score(bclf, X, y, cv=5, scoring='f1')
print(scores.mean())

nan


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rclf = RandomForestClassifier(random_state=2020, criterion='gini', max_depth=3, min_samples_split=3,
                             n_estimators=120, max_features=0.9, class_weight='balanced')
scores = cross_val_score(rclf, X, y, cv=4, scoring='f1')
print(scores.mean())

nan


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
aclf = AdaBoostClassifier(n_estimators=50)
scores = cross_val_score(aclf, X, y, cv=5, scoring='f1')
print(scores.mean())

nan


In [ ]:
pred1 = dt.predict(test)
pred2 = nb.predict(test)
pred3 = kn.predict(test)

pred = (pred1+pred2+pred3)/3 # averaging
pred = pred1*0.2 + pred2*0.4 + pred3*0.4 # weighted averaging

In [ ]:
iris = datasets.load_iris()
X, y = iris.data[:, 1:3], iris.target

In [ ]:
lr = LogisticRegression()
sclf = StackingCVClassifier(classifiers=[clf1, clf2, clf3], meta_classifier=lr)
scores = cross_val_score(sclf, X, y, cv=5, scoring='accuracy')
print(scores.mean())

0.9533333333333334


In [ ]:
df = pd.read_csv('/content/train_insurance.csv')
df.head()

,perc_premium_paid_by_cash_credit,age_in_days,Income,Count_3.6_months_late,Count_6.12_months_late,Count_more_than_12_months_late,application_underwriting_score,no_of_premiums_paid,sourcing_channel,residence_area_type,premium,renewal
0,0.429,12058,355060,0.0,0.0,0.0,99.02,13,C,Urban,3300,1
1,0.917,17531,84140,2.0,3.0,1.0,98.69,7,C,Rural,3300,0
2,0.049,15341,250510,0.0,0.0,0.0,99.57,9,A,Urban,9600,1
3,0.052,31400,198680,0.0,0.0,0.0,99.87,12,B,Urban,9600,1
4,1.000,24829,118400,0.0,0.0,0.0,99.05,11,B,Urban,7500,1


In [ ]:
X = df.drop('renewal', axis=1)
y = df['renewal']
y.value_counts()

1    49933
0     3303
Name: renewal, dtype: int64

In [ ]:
# X = X.fillna(-1)
X.isnull().sum()

perc_premium_paid_by_cash_credit       0
age_in_days                            0
Income                                 0
Count_3.6_months_late                 64
Count_6.12_months_late                64
Count_more_than_12_months_late        64
application_underwriting_score      1976
no_of_premiums_paid                    0
sourcing_channel                       0
residence_area_type                    0
premium                                0
dtype: int64

In [ ]:
LE = LabelEncoder()
for c in X.columns:
  if X[c].dtypes=='object':
    X[c] = LE.fit_transform(X[c])

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, y, test_size=0.3) # 70 15 15
Xtest, Xvalid, Ytest, Yvalid = train_test_split(Xtest, Ytest, test_size=0.5)
print(Ytrain.shape, Ytest.shape, Yvalid.shape)

(37265,) (7985,) (7986,)


In [ ]:
import xgboost

XGB = xgboost.XGBClassifier(random_state=2020, n_estimators=2000, max_depth=8, colsample_bytree=0.85, missing=None,
                            scale_pos_weight=0.1, learning_rate=0.008, reg_alpha=2.01, reg_lambda=0.05, verbosity=1)

XGB.fit(Xtrain, Ytrain, eval_set=[(Xtrain, Ytrain),(Xvalid, Yvalid)], verbose=2, early_stopping_rounds=10)
pred = XGB.predict(Xtest)
f1_score(Ytest, pred, pos_label=0)

[0]	validation_0-error:0.192191	validation_1-error:0.200726
Multiple eval metrics have been passed: 'validation_1-error' will be used for early stopping.

Will train until validation_1-error hasn't improved in 10 rounds.
[2]	validation_0-error:0.138307	validation_1-error:0.146632
[4]	validation_0-error:0.133262	validation_1-error:0.140997
[6]	validation_0-error:0.131222	validation_1-error:0.138367
[8]	validation_0-error:0.124943	validation_1-error:0.129351
[10]	validation_0-error:0.128029	validation_1-error:0.133484
[12]	validation_0-error:0.127251	validation_1-error:0.132607
[14]	validation_0-error:0.128754	validation_1-error:0.13436
[16]	validation_0-error:0.130149	validation_1-error:0.136113
[18]	validation_0-error:0.129451	validation_1-error:0.133609
Stopping. Best iteration:
[8]	validation_0-error:0.124943	validation_1-error:0.129351



0.3774792066538708

In [ ]:
import lightgbm

lgb = lightgbm.LGBMClassifier(random_state=2020, n_estimators=2000, num_leaves=31,
                              learning_rate=0.011, colsample_bytree=0.85, max_depth=5,
                              verbose=1, class_weight={0:0.2, 1:0.8})
lgb.fit(Xtrain, Ytrain, eval_set=(Xvalid, Yvalid), verbose=100, early_stopping_rounds=50)
pred = lgb.predict(Xtest)
print(f1_score(Ytest, pred, pos_label=0))

Training until validation scores don't improve for 50 rounds.
[100]	valid_0's binary_logloss: 0.221014
[200]	valid_0's binary_logloss: 0.213394
[300]	valid_0's binary_logloss: 0.209699
[400]	valid_0's binary_logloss: 0.206996
[500]	valid_0's binary_logloss: 0.205109
[600]	valid_0's binary_logloss: 0.203445
[700]	valid_0's binary_logloss: 0.201829
[800]	valid_0's binary_logloss: 0.200112
[900]	valid_0's binary_logloss: 0.198108
[1000]	valid_0's binary_logloss: 0.196241
[1100]	valid_0's binary_logloss: 0.194546
[1200]	valid_0's binary_logloss: 0.193135
[1300]	valid_0's binary_logloss: 0.191656
[1400]	valid_0's binary_logloss: 0.19016
[1500]	valid_0's binary_logloss: 0.188879
[1600]	valid_0's binary_logloss: 0.187114
[1700]	valid_0's binary_logloss: 0.185622
[1800]	valid_0's binary_logloss: 0.18421
[1900]	valid_0's binary_logloss: 0.182823
[2000]	valid_0's binary_logloss: 0.181686
Did not meet early stopping. Best iteration is:
[2000]	valid_0's binary_logloss: 0.181686
0.05924170616113743

In [ ]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 55 kB/s 


In [ ]:
from catboost import CatBoostClassifier
cat_features = ['sourcing_channel', 'residence_area_type']
model = CatBoostClassifier(iterations=200, learning_rate=0.1, depth=5, cat_features=cat_features)
model.fit(Xtrain, Ytrain, eval_set=(Xtest, Ytest), verbose=10, early_stopping_rounds=50)
pred = model.predict(Xtest)
f1_score(pred, Ytest, pos_label=0)

0:	learn: 0.5718555	test: 0.5716168	best: 0.5716168 (0)	total: 82.9ms	remaining: 16.5s
10:	learn: 0.2126967	test: 0.2117345	best: 0.2117345 (10)	total: 447ms	remaining: 7.68s
20:	learn: 0.1842447	test: 0.1845016	best: 0.1845016 (20)	total: 792ms	remaining: 6.75s
30:	learn: 0.1790397	test: 0.1802562	best: 0.1802562 (30)	total: 1.54s	remaining: 8.42s
40:	learn: 0.1770372	test: 0.1791959	best: 0.1791959 (40)	total: 1.96s	remaining: 7.59s
50:	learn: 0.1759129	test: 0.1786755	best: 0.1786755 (50)	total: 2.32s	remaining: 6.79s
60:	learn: 0.1749667	test: 0.1786394	best: 0.1786394 (60)	total: 2.67s	remaining: 6.09s
70:	learn: 0.1742051	test: 0.1785876	best: 0.1785696 (69)	total: 3.02s	remaining: 5.5s
80:	learn: 0.1733705	test: 0.1785241	best: 0.1785241 (80)	total: 3.54s	remaining: 5.2s
90:	learn: 0.1726733	test: 0.1785603	best: 0.1785081 (83)	total: 3.94s	remaining: 4.72s
100:	learn: 0.1720429	test: 0.1784771	best: 0.1784771 (100)	total: 4.31s	remaining: 4.22s
110:	learn: 0.1712666	test: 0.178

0.19839679358717432